#Stock Indicators Python

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
import yfinance as yf
df = yf.download('SPY', start='2020-01-01', multi_level_index=False)
df.reset_index(inplace=True)
rawdf = df.copy()
df.head()

C:\Users\jwang\AppData\Local\Temp\ipykernel_18900\453595078.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('SPY', start='2020-01-01', multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


,Date,Close,High,Low,Open,Volume
0,2020-01-02,298.578705,298.597104,296.428081,297.356352,59151200
1,2020-01-03,296.317719,297.448186,295.113738,295.168880,77709700
2,2020-01-06,297.448212,297.530925,294.433631,294.553115,55653900
3,2020-01-07,296.611847,297.356292,296.161483,296.878357,40496400
4,2020-01-08,298.192688,299.415040,296.556742,296.804881,68296000


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]


In [4]:

# Calculate EMA
df['EMA 12'] = [r.ema for r in indicators.get_ema(quotes, 12)]
df['EMA 20'] = [r.ema for r in indicators.get_ema(quotes, 20)]
df['EMA 25'] = [r.ema for r in indicators.get_ema(quotes, 25)]

# Calculate EMA12 crossover EMA25
df['bullishEMA'] = 0.0
df['bullishEMA'] = np.where(df['EMA 12'] > df['EMA 25'], 1.0, 0.0)
df['crossover_EMA12_EMA25'] = df['bullishEMA'].diff()

# Calculate Chandelier exit
from stock_indicators import ChandelierType 
df['chandelier_long_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.LONG)]
df['chandelier_short_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.SHORT)]

# Calculate Bollinger Bands
df['upper_band'] = [r.upper_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['middle_band'] = [r.sma for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['lower_band'] = [r.lower_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]

# Calculate RSI
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df['rsima6'] = df['rsi'].rolling(6).mean()
df['rsima12'] = df['rsi'].rolling(12).mean()

# Calculate Mcginley dynamic
df['dynamic20'] = [r.dynamic for r in indicators.get_dynamic(quotes, 20)]


In [5]:
df = df.dropna().reset_index(drop=True)
df.tail()

,Date,Close,High,Low,Open,Volume,EMA 12,EMA 20,EMA 25,bullishEMA,crossover_EMA12_EMA25,chandelier_long_exit,chandelier_short_exit,upper_band,middle_band,lower_band,rsi,rsima6,rsima12,dynamic20
1426,2025-10-10,653.020020,673.950012,652.840027,672.130005,159422600,665.273432,662.753321,660.839245,1.0,0.0,656.842757,668.890275,675.082525,664.226224,653.369923,42.498581,64.535698,65.006460,660.783553
1427,2025-10-13,663.039978,665.130005,659.770020,660.650024,79560500,664.929824,662.780622,661.008532,1.0,0.0,655.968998,670.472045,675.039781,664.424088,653.808394,52.632831,61.696228,64.458754,660.969042
1428,2025-10-14,662.229980,665.830017,653.169983,657.169983,88779600,664.514463,662.728180,661.102490,1.0,0.0,655.059948,671.381095,674.906986,664.626825,654.346664,51.837479,58.364143,63.524600,661.073322
1429,2025-10-15,665.169983,670.229980,658.929993,666.820007,81702600,664.615312,662.960733,661.415374,1.0,0.0,654.377680,672.063363,674.741741,665.017447,655.293153,54.523669,56.362266,62.667550,661.406378
1430,2025-10-16,660.640015,668.710022,657.109985,666.820007,110409600,664.003728,662.739712,661.355731,1.0,0.0,653.685508,672.755535,674.732794,665.027997,655.323200,49.905127,52.961186,61.236384,661.342217


In [6]:
# Now, let's create the chart with lightweight-charts and add the EMAs and buy/sell markers.

# Assuming `df` is already a pandas DataFrame with 'Date', 'Open', 'High', 'Low', 'Close', 'EMA 12', and 'EMA 25' columns.
# It's good practice to convert the 'Date' column to the correct datetime format.
# df['Date'] = pd.to_datetime(df['Date'])

if __name__ == '__main__':
    
    rt_chart = Chart( maximize=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    rt_chart.set(df)

    # Create line series for EMAs
    ema12_line = rt_chart.create_line('EMA 12', color='#ffeb3b', width=1, price_label=True)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = rt_chart.create_line('EMA 25', color='#26c6da', width=1, price_label=True)
    ema25_line.set(df[['Date', 'EMA 25']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        emadiff = df.iloc[i]['crossover_EMA12_EMA25']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if emadiff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif emadiff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        rt_chart.marker_list(markers)
    
rt_chart.show(block = True)
